# Module 1, Section 1: Foundational Concepts

In this section, we'll build a simple customer support system for TechHub, a fictional e-commerce electronics store. 

We'll start with the basics:
1. How to make LLM calls with messages
2. How to define tools that access external data (our TechHub database)
3. The manual tool calling loop (involved but educational!)

By the end, you'll understand **why** agent frameworks exist - spoiler, they automate the tedious parts we're about to implement manually

## Setup

First, let's load our environment variables (API keys).


In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## 1. Basic LLM Call

Let's start simple - just calling an LLM with a message.

**Note:** The default model is set workshop-wide via WORKSHOP_MODEL in the .env file (see config.py).


In [2]:
from langchain.chat_models import init_chat_model
from config import DEFAULT_MODEL

print(f"Using model: {DEFAULT_MODEL}")

# Initialize the model (using workshop default)
llm = init_chat_model(DEFAULT_MODEL)

# Simple string input
response = llm.invoke("What is LangChain in under 10 words?")
response.pretty_print()

Using model: anthropic:claude-haiku-4-5
================================== Ai Message ==================================

Framework for building applications with large language models.


## 2. Working with Messages

LLMs work with **messages** that have roles:
- `SystemMessage`: Instructions for how the LLM should behave
- `HumanMessage`: User input
- `AIMessage`: Model responses
- `ToolMessage`: Results from tool execution (we'll see this soon!)


In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Multi-turn conversation with messages
messages = [
    SystemMessage(
        content="You are a helpful customer support assistant for TechHub, an electronics store."
    ),
    HumanMessage(content="Hello!"),
    AIMessage(content="Hi! How can I help you today?"),
    HumanMessage(content="What's the status of order ORD-2024-0123?"),
]

response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

Hello! I'd be happy to help you check on your order, but I don't have access to our order management system or customer database.

To check your order status for **ORD-2024-0123**, here are a few ways I can help you get that information:

1. **Check your email** - You should have received a confirmation email with tracking information
2. **Visit our website** - Log into your account and go to "My Orders"
3. **Contact our support team directly** - They can look up your order using:
   - Your order number (ORD-2024-0123)
   - Your account email or phone number

Is there anything else I can help you with, like answering questions about our products or policies?


**Notice the problem:** The LLM can't actually look up the order! It doesn't have access to our database.

This is where **tools** come in.


## 3. Defining Tools

<div align="center">
    <img src="../../static/db_tools.png" alt="Schema Diagram">
</div>

Tools give LLMs the ability to interact with external systems. Let's create three simple tools that query our TechHub database.

The `@tool` decorator automatically:
- Extracts the function signature for the LLM
- Uses the docstring as the tool description
- Handles the input/output formatting


In [21]:
import sqlite3
from pathlib import Path
from langchain.tools import tool

# Path to our TechHub database
DB_PATH = Path("../../data/structured/techhub.db")


@tool
def get_order_status(order_id: str) -> str:
    """Get status, dates, and tracking information for a specific order.

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with order status, dates, tracking number, and total amount.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute(
        """
        SELECT order_id, order_date, status, shipped_date, tracking_number, total_amount
        FROM orders
        WHERE order_id = ?
    """,
        (order_id,),
    )

    result = cursor.fetchone()
    conn.close()

    if not result:
        return f"Order {order_id} not found."

    order_id, order_date, status, shipped_date, tracking_number, total = result

    response = f"Order {order_id}:\n"
    response += f"- Status: {status}\n"
    response += f"- Order Date: {order_date}\n"
    response += f"- Total Amount: ${total:.2f}\n"

    if shipped_date:
        response += f"- Shipped Date: {shipped_date}\n"
    if tracking_number:
        response += f"- Tracking Number: {tracking_number}\n"

    return response


@tool
def get_order_items(order_id: str) -> str:
    """Get list of items in a specific order with product IDs, quantities, and prices.

    Args:
        order_id: The order ID (e.g., "ORD-2024-0123")

    Returns:
        Formatted string with product IDs, quantities, and prices for each item.
        Note: Returns product IDs only - use get_product_info() to get product names.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute(
        """
        SELECT product_id, quantity, price_per_unit
        FROM order_items
        WHERE order_id = ?
    """,
        (order_id,),
    )

    items = cursor.fetchall()
    conn.close()

    if not items:
        return f"No items found for order {order_id}."

    response = f"Items in order {order_id}:\n"
    for product_id, quantity, price in items:
        response += (
            f"- Product ID: {product_id}, Quantity: {quantity}, Price: ${price:.2f}\n"
        )

    return response


@tool
def get_product_info(product_identifier: str) -> str:
    """Get product details by product name or product ID.

    Args:
        product_identifier: Product name (e.g., "MacBook Air") or product ID (e.g., "TECH-LAP-001")

    Returns:
        Formatted string with product name, category, price, and stock status.
    """
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Try exact ID match first
    cursor.execute(
        """
        SELECT product_id, name, category, price, in_stock
        FROM products
        WHERE product_id = ?
    """,
        (product_identifier,),
    )

    result = cursor.fetchone()

    # If no exact match, try fuzzy name search
    if not result:
        cursor.execute(
            """
            SELECT product_id, name, category, price, in_stock
            FROM products
            WHERE name LIKE ?
            LIMIT 1
        """,
            (f"%{product_identifier}%",),
        )

        result = cursor.fetchone()

    conn.close()

    if not result:
        return f"Product '{product_identifier}' not found."

    product_id, name, category, price, in_stock = result
    stock_status = "In Stock" if in_stock else "Out of Stock"

    return f"{name} ({product_id})\n- Category: {category}\n- Price: ${price:.2f}\n- Status: {stock_status}"

Let's test out the `get_order_status` tool

In [22]:
example_order = "ORD-2024-0127"
result = get_order_status.invoke(example_order)
print(result)

Order ORD-2024-0127:
- Status: Delivered
- Order Date: 2024-12-12
- Total Amount: $989.00
- Shipped Date: 2024-12-15
- Tracking Number: 1Z999AA174245343



Let's inspect how the `@tool` decorator parses the tool information into a `StructuredTool` object.


In [ ]:
print("\n--- TOOL NAME ---")
print(get_order_status.name)
print("\n--- TOOL DESCRIPTION ---")
print(get_order_status.description)
print("\n--- TOOL ARGUMENTS ---")
print(get_order_status.args)


--- TOOL NAME ---
get_order_status

--- TOOL DESCRIPTION ---
Get status, dates, and tracking information for a specific order.

Args:
    order_id: The order ID (e.g., "ORD-2024-0123")

Returns:
    Formatted string with order status, dates, tracking number, and total amount.

--- TOOL ARGUMENTS ---
{'order_id': {'title': 'Order Id', 'type': 'string'}}


## 4. The Manual Tool Calling Loop

Now let's see how LLMs actually use tools! This happens in stages:

1. **Give the LLM access to tools** - Bind tools so the LLM knows what's available
2. **LLM decides when to call a tool** - Based on tool descriptions and the user's query
3. **LLM formats the function call** - Returns which tool to call and with what arguments (but doesn't execute it!)
4. **We manually execute the tool** - Run the actual function to get results
5. **Pass results back to the LLM** - So it can use them to generate a final answer

Let's see each stage in action!


### Step 1: Bind Tools to the Model

First, we tell the LLM what tools are available by "binding" them:


In [11]:
# Bind tools to the model - this tells the LLM what tools are available
tools = [get_order_status, get_order_items, get_product_info]
llm_with_tools = llm.bind_tools(tools)

### Step 2: LLM Decides to Call a Tool

Now let's give the LLM a query. It will decide which tool to call based on the descriptions:


In [12]:
# Ask about an order
query = "What's the status of order ORD-2024-0123?"

messages = [
    SystemMessage(content="You are a helpful customer support assistant."),
    HumanMessage(content=query),
]

# Call the LLM
response = llm_with_tools.invoke(messages)

# Add the AI's tool call to messages
messages.append(response)

response.pretty_print()

================================== Ai Message ==================================

[{'id': 'toolu_01EAqdxa9ZkSgZdFAvmCgEBP', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}]
Tool Calls:
  get_order_status (toolu_01EAqdxa9ZkSgZdFAvmCgEBP)
 Call ID: toolu_01EAqdxa9ZkSgZdFAvmCgEBP
  Args:
    order_id: ORD-2024-0123


### Step 3: Manually Execute the Tool

The LLM told us WHAT to call and WITH WHAT ARGUMENTS. Now **we** need to actually run the function:


In [13]:
# Extract the tool call information
tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]

print(f"[Executing]\n {tool_name}(**{tool_args})", "\n")

# Manually execute the tool
if tool_name == "get_order_status":
    tool_result = get_order_status.invoke(tool_args)
elif tool_name == "get_order_items":
    tool_result = get_order_items.invoke(tool_args)
elif tool_name == "get_product_info":
    tool_result = get_product_info.invoke(tool_args)

print(f"[Tool Result]\n {tool_result}")

[Executing]
 get_order_status(**{'order_id': 'ORD-2024-0123'}) 

[Tool Result]
 Order ORD-2024-0123:
- Status: Delivered
- Order Date: 2024-12-05
- Total Amount: $129.00
- Shipped Date: 2024-12-07
- Tracking Number: 1Z999AA113527782



### Step 4: Pass Results Back to the LLM

Now we need to send the tool's result back to the LLM so it can generate a final answer for the user. Note that we use a `ToolMessage` type when adding tool result details back to the message history.


In [14]:
from langchain_core.messages import ToolMessage

# Create a ToolMessage with the result
tool_message = ToolMessage(
    content=str(tool_result),
    tool_call_id=tool_call["id"],  # Must match the ID from the tool call
)
messages.append(tool_message)
messages

[SystemMessage(content='You are a helpful customer support assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="What's the status of order ORD-2024-0123?", additional_kwargs={}, response_metadata={}),
 AIMessage(content=[{'id': 'toolu_01EAqdxa9ZkSgZdFAvmCgEBP', 'input': {'order_id': 'ORD-2024-0123'}, 'name': 'get_order_status', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_012rkME6nJUWZEkgcAAkSA4Y', 'model': 'claude-haiku-4-5-20251001', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation': {'ephemeral_1h_input_tokens': 0, 'ephemeral_5m_input_tokens': 0}, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 928, 'output_tokens': 66, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-haiku-4-5-20251001', 'model_provider': 'anthropic'}, id='lc_run--1fcb614a-710f-47e9-bd03-8fcdc78dfd97-0', tool_calls=[{'name': 'get_order_status', 'args': {'order_id': 'ORD-2024

### Step 5: Get the Final Answer

Now the LLM has the tool result and can give a complete answer to the user:


In [15]:
# Call the LLM again with the tool result
final_response = llm_with_tools.invoke(messages)

final_response.pretty_print()

================================== Ai Message ==================================

Great! Here's the status of your order:

**Order ORD-2024-0123**
- **Status:** Delivered ✓
- **Order Date:** December 5, 2024
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782
- **Total Amount:** $129.00

Your order has been successfully delivered! If you need any other information about this order or have any questions, feel free to ask.


### Putting It All Together: The Loop

In real scenarios, the LLM might need to call multiple tools or loop several times. Let's create a function that automates this process:


In [16]:
def run_agent_loop(user_query: str):
    """Run the complete tool calling loop."""

    messages = [
        SystemMessage(
            content="You are a helpful customer support assistant for TechHub."
        ),
        HumanMessage(content=user_query),
    ]

    print(f"\n{'='*60}")
    print(f"User: {user_query}")
    print(f"{'='*60}\n")

    # Keep looping until we get a final answer
    iteration = 0
    while True:
        iteration += 1

        # Call the LLM
        response = llm_with_tools.invoke(messages)

        # Check if LLM wants to use tools
        if not response.tool_calls:
            # No tools needed - we have the final answer!
            print(f"[Final Answer]\n {response.content}\n")
            break

        # LLM wants to use tools
        print(
            f"[Iteration {iteration}] LLM is calling {len(response.tool_calls)} tool(s)..."
        )
        messages.append(response)

        # Execute each tool
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            tool_args = tool_call["args"]

            # Execute the tool
            if tool_name == "get_order_status":
                result = get_order_status.invoke(tool_args)
            elif tool_name == "get_order_items":
                result = get_order_items.invoke(tool_args)
            elif tool_name == "get_product_info":
                result = get_product_info.invoke(tool_args)

            print(f"  → {tool_name}({list(tool_args.values())[0]}) = {result[:50]}...")

            # Add tool result to messages
            messages.append(
                ToolMessage(content=str(result), tool_call_id=tool_call["id"])
            )

        print()  # Blank line before next iteration

### Try It Out!

Now let's test our agent loop with different queries:


**Example 1: Simple order lookup**


In [17]:
run_agent_loop("What's the status of order ORD-2024-0123?")


User: What's the status of order ORD-2024-0123?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_order_status(ORD-2024-0123) = Order ORD-2024-0123:
- Status: Delivered
- Order D...

[Final Answer]
 Great news! Here's the status of your order **ORD-2024-0123**:

- **Status**: ✓ Delivered
- **Order Date**: December 5, 2024
- **Shipped Date**: December 7, 2024
- **Tracking Number**: 1Z999AA113527782
- **Total Amount**: $129.00

Your order has been successfully delivered! If you need any additional information about the items in this order or have any other questions, feel free to ask.



**Example 2: Product price lookup**


In [18]:
run_agent_loop("How much does the MacBook Air cost?")


User: How much does the MacBook Air cost?

[Iteration 1] LLM is calling 1 tool(s)...
  → get_product_info(MacBook Air) = MacBook Air M2 (13-inch, 256GB) (TECH-LAP-001)
- C...

[Final Answer]
 The **MacBook Air M2 (13-inch, 256GB)** costs **$1,199.00** and is currently in stock. This is our product ID TECH-LAP-001 in the Laptops category.

Is there anything else you'd like to know about this MacBook Air or any other products?



**Example 3: Multiple tools in one query**


In [20]:
run_agent_loop(
    "What's the status of order ORD-2024-0123 and how much does a MacBook Air cost?"
)


User: What's the status of order ORD-2024-0123 and how much does a MacBook Air cost?

[Iteration 1] LLM is calling 2 tool(s)...
  → get_order_status(ORD-2024-0123) = Order ORD-2024-0123:
- Status: Delivered
- Order D...
  → get_product_info(MacBook Air) = MacBook Air M2 (13-inch, 256GB) (TECH-LAP-001)
- C...

[Final Answer]
 Here's the information you requested:

**Order ORD-2024-0123 Status:**
- **Status:** Delivered
- **Order Date:** December 5, 2024
- **Shipped Date:** December 7, 2024
- **Tracking Number:** 1Z999AA113527782
- **Total Amount:** $129.00

**MacBook Air Pricing:**
- **Product:** MacBook Air M2 (13-inch, 256GB)
- **Price:** $1,199.00
- **Availability:** In Stock

Is there anything else you'd like to know about your order or our products?



## Key Takeaways

**What We Learned**

1. **LLMs can't access external data** without tools
2. **Tools are functions** with clear schemas (name, description, arguments)
3. **The tool calling loop** is: LLM → tool call → execute → result → LLM → repeat until done
4. **This is tedious!** We had to:
   - Manually check for tool calls
   - Execute each tool ourselves
   - Format and append results
   - Manage the loop logic

**Why This Matters**

Understanding the manual loop is crucial because:
- **Every agent framework does this** under the hood
- You can debug agent behavior by understanding what's happening
- You'll appreciate the abstractions in Section 2!

**What's Next**

In **Section 2**, we'll replace all this manual code with `create_agent` - a simple abstraction that:
- Handles the tool calling loop automatically
- Adds memory (conversation history)
- Supports streaming responses
- Requires just a few lines of code!
